In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from scipy import interpolate
from scipy import integrate
from astropy import constants as constants
from astropy import units as u
%matplotlib inline

In [3]:
ms_table_file = 'ekert_2018_table_7_MS.dat'
dat = ascii.read(ms_table_file)

In [4]:
# Names of columns in the ascii table
print dat.colnames

['Spt', 'logTeff', 'B_V', 'U_B', 'BC', 'MV', 'Teff', 'M', 'R', 'logg', 'M_over_L', 'L_over_M']


In [5]:
# Functions to interpolate Teff, luminosities, as a function of stellar mass
Teff_vs_M = interpolate.interp1d(dat['M'], dat['Teff'], 
                                 fill_value='extrapolate')
logTeff_vs_M = interpolate.interp1d(dat['M'], dat['logTeff'], 
                                 fill_value='extrapolate')
L_over_M_vs_M = interpolate.interp1d(dat['M'], dat['L_over_M'], 
                                 fill_value='extrapolate')

In [6]:
# set up properties of the IMF
Mmin = 0.08
Mmax = 120.
slope = -2.35

# function to generate a power law initial mass function initialized
# to one solar mass when integrated between Mmin and Mmax
#
# https://en.wikipedia.org/wiki/Initial_mass_function
#
def imf_power_law(M, slope=slope, Mmin=Mmin, Mmax=Mmax):
    # normalization 
    norm = 1.0 / ((1.0/(slope+1))*(Mmax**(slope+1) - Mmin**(slope+1)))
    imf = norm * M**slope
    return imf

# check normalization
#print integrate.quad(imf_power_law,
#                     Mmin, Mmax, args=(slope))



In [7]:
# Set up vector of Masses, Temperatures, bolometric Luminosities, 
# and IMF weighting
#
# Units: Solar Masses, Kelvin, Solar Luminosities, inverse solar masses
npoints = 200
lgM = np.linspace(np.log10(Mmin),np.log10(Mmax),npoints)
M = 10**lgM
T = Teff_vs_M(M)
Lbol = L_over_M_vs_M(M) * M
imf = imf_power_law(M)

In [8]:
# Function to return a black body spectrum for wavelength in angstroms
# and temperature in Kelvin. Output in cgs units.
#
# This can be integrated using:
# integrate.quad(blackbody_wavelength_angstroms,
#                lambda_min, lambda_max,
#                args=(temperature))
# which returns a 2-element tuple where the first value contains the definite integral

def blackbody_wavelength_angstroms(lam, temperature):
    _h = constants.h.cgs.value
    _c = constants.c.cgs.value
    _k_B = constants.k_B.cgs.value
    
    I = (2.0*_h*_c**2 / lam**5 * (np.exp(_h*_c/(_k_B*temperature*(lam*1.e-8))) 
                                  - 1.)**-1)

    return I

In [9]:
# function to return the log of the main sequence lifetime 
# as a function of stellar mass

def ms_lifetime_log_yrs(M):
    lgt = (10.09 - 3.139*np.log10(M)
                   + 0.238238*np.log10(M)**2
                   + 0.26163378*np.log10(M)**3)
    # > 8 Msun
    lgt_hi = (9.01 - 1.57*np.log10(M))
    i = np.where(M>20)[0]
    lgt[i]=lgt_hi[i]
    return lgt